# SEIR Scaling Study
Series of simulation to study how the virus behave at different scales. We will study how it does respond to different initial conditions, and to different total population. We will study Seroprevalence, Peak size and peak date. 

## Results:
From this simulations we can conclude that the SEIR models are free of scale, their behavior depends only in the ratio of initial infected to the total population.

## Import Libraries

In [9]:
import sys
from pathlib import Path
sys.path.insert(1, '../src/SEIR/')
sys.path.insert(1, '../src/utils/')

from datetime import datetime
import numpy as np
from numpy import linalg as LA
import pandas as pd
from time import time
import multiprocessing
from joblib import Parallel, delayed

import platform
OS = platform.system()

import matplotlib.pyplot as plt
if OS == 'Linux':    
    %matplotlib tk
    print('Linux')
elif OS == 'Windows':
    %matplotlib qt
    print('Windows')
else:
    print('OS not detected :-|')


from class_SEIR2 import SEIR
from Quarantine import Quarantine

Linux


## Epidemiological Parameters
* **beta:** Infection rate
* **mu:** Initial exposed obtained from the initial infected mu=E0/I0
* **Scale Factor:** Proportion of real infected compared to reported ones (1: all the infecteds are reported)
* **Sero Prevalence Factor:** Adjust the proportion of the population that enters the virus dynamics
* **Exposed Infection:** rate compared to the infected (0 the don't infect, 1 the infect in the same rate as the infected )

## Simulation Parameters

In [5]:
beta = 0.15 # Contagion rate
mu = 0 # E0/I0 initial rate
sigma = 0.2
gamma = 0.1


# Simulation time
tsim = 1000

# No quarantine
s1 = Quarantine(1)
quarantine = s1.alpha

In [6]:
# Population Conditions
population = [1000,10000,100000,1000000]
# Relative Initial Active Infected 
I0 = [0.01,0.05,0.1,0.5,1,5,10]

## Create and run simulation

In [10]:
# For parallel simulation
def simulate(simulation,j,tsim):
    simulation[j].integr_sci(0,tsim,0.1)
    return simulation[j]

# Define Simulations
sims = []
for i in range(len(population)):
    aux = []
    for j in range(len(I0)):
        aux.append(SEIR(tsim=tsim,alpha=s1.alpha,beta=beta,mu=mu,I0=I0[j]*population[i],population=population[i],expinfection=0,SeroPrevFactor=1))
    sims.append(aux)
# Run simulation
num_cores = multiprocessing.cpu_count()
simulation = []
for i in range(len(population)):
    simulation.append(Parallel(n_jobs=num_cores, verbose=50)(delayed(simulate)(sims[i],j,tsim) for j in range(len(sims[i]))))

print('ready')

[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   1 tasks      | elapsed:    0.9s
[Parallel(n_jobs=12)]: Done   2 out of   7 | elapsed:    0.9s remaining:    2.2s
[Parallel(n_jobs=12)]: Done   3 out of   7 | elapsed:    0.9s remaining:    1.2s
[Parallel(n_jobs=12)]: Done   4 out of   7 | elapsed:    0.9s remaining:    0.7s
[Parallel(n_jobs=12)]: Done   5 out of   7 | elapsed:    0.9s remaining:    0.4s
[Parallel(n_jobs=12)]: Done   7 out of   7 | elapsed:    1.1s remaining:    0.0s
[Parallel(n_jobs=12)]: Done   7 out of   7 | elapsed:    1.1s finished
[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   1 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Batch computation too fast (0.0412s.) Setting batch_size=2.
[Parallel(n_jobs=12)]: Done   2 out of   7 | elapsed:    0.1s remaining:    0.2s
[Parallel(n_jobs=12)]: Done   3 out of   7 | elapsed:    0.6s remaining:    0.8s
[

# Simulation Analysis

In [12]:
# Calculate indicators
peak = []
for i in range(len(population)):
    aux = []
    for j in range(len(I0)):
        aux.append(100*simulation[i][j].peak/population[i])
    peak.append(aux)

    
peaktime = []
for i in range(len(population)):
    aux = []
    for j in range(len(I0)):
        aux.append(simulation[i][j].peak_t)
    peaktime.append(aux)

prevalence = []
for i in range(len(population)):
    aux = []
    for j in range(len(I0)):    
        aux.append(simulation[i][j].prevalence_total[-1])  
    prevalence.append(aux)

### Peak Size Proportion

In [17]:
# Contour Plot
fig,ax=plt.subplots(1,1)
cp = ax.contourf(I0,population,peak) 
fig.colorbar(cp) # Add a colorbar to a plot
ax.set_title('Peak Size Proportion')
ax.set_xlabel('Initial Active Infected')
ax.set_ylabel('Total Population')
plt.show() 

### Peak time

In [19]:
# Contour Plot
fig,ax=plt.subplots(1,1)
cp = ax.contourf(I0,population,peaktime) 
fig.colorbar(cp) # Add a colorbar to a plot
ax.set_title('Peak Time')
ax.set_xlabel('Initial Active Infected')
ax.set_ylabel('Total Population')
plt.show() 

### SeroPrevalence

In [20]:
fig,ax=plt.subplots(1,1)
cp = ax.contourf(I0,population,prevalence) 
fig.colorbar(cp) # Add a colorbar to a plot
ax.set_title('Prevalence')
ax.set_xlabel('Initial Active Infected')
ax.set_ylabel('Total Population')
plt.show() 